In [5]:
from nltk.translate.bleu_score import sentence_bleu
from rouge_score import rouge_scorer
from bert_score import score as bert_score
from rouge import Rouge
import numpy as np

In [7]:
references = [
    "Op basis van de informatie uit de vergelijkbare zaak wordt de verdachte vrijgesproken van het eerste ten laste gelegde feit en schuldig bevonden aan het tweede ten laste gelegde feit, namelijk bedreiging van ambtenaren tijdens de rechtmatige uitoefening van hun bediening. De verdachte wordt veroordeeld tot een gevangenisstraf van 6 maanden. Deze straf wordt bepaald op basis van de ernst van de feiten en de persoonlijke omstandigheden van de verdachte. Daarnaast kan er een contactverbod met de slachtoffers en eventueel een schadevergoeding worden opgelegd."] * 49

hypotheses = [
    "Op basis van de gegeven context en tenlastelegging wordt de verdachte schuldig bevonden aan het ten laste gelegde feit en wordt als volgt veroordeeld:\nFeit 1: Veroordeeld tot een gevangenisstraf van 2 maanden voor het toebrengen van zwaar lichamelijk letsel of mishandeling aan [slachtoffer].\nFeit 2: Veroordeeld tot een taakstraf van 240 uren en een voorwaardelijke gevangenisstraf van 12 maanden voor het bedreigen van ambtenaren met een misdrijf tegen het leven gericht.\nDe specifieke straffen kunnen variëren afhankelijk van de omstandigheden en het verdere verloop van de rechtszaak.",
    "Op basis van de verstrekte context en de uitspraak in een vergelijkbare zaak wordt de verdachte schuldig bevonden aan mishandeling en veroordeeld tot een gevangenisstraf van 6 maanden, met een proeftijd van 3 jaar en een taakstraf van 160 uren. Indien de taakstraf niet naar behoren wordt uitgevoerd, zal vervangende hechtenis van 80 dagen worden toegepast. De tijd doorgebracht in voorarrest zal in mindering worden gebracht op de taakstraf.",
    "Op basis van de verstrekte context en de uitspraak in een vergelijkbare zaak waarbij de verdachte schuldig was bevonden aan bedreiging van ambtenaren in functie, wordt de verdachte schuldig bevonden aan bedreiging met enig misdrijf tegen het leven gericht en/of met zware mishandeling van ambtenaren in functie. De verdachte wordt veroordeeld tot een gevangenisstraf van 18 maanden. De tijd doorgebracht in voorarrest zal in mindering worden gebracht op de gevangenisstraf.",
    "Op basis van de ontvangen context wordt de verdachte schuldig bevonden aan bedreiging van ambtenaren in functie en mishandeling van het slachtoffer. De verdachte wordt veroordeeld tot een gevangenisstraf van 18 maanden, waarbij de tijd in voorarrest in mindering zal worden gebracht op de straf.",
    "Op basis van de ontvangen informatie uit het vergelijkbare geval wordt de verdachte schuldig bevonden aan mishandeling voor het opzettelijk snijden in het gezicht van het slachtoffer met een mes. De verdachte wordt veroordeeld tot een gevangenisstraf van 6 maanden, waarbij de tijd in voorarrest in mindering zal worden gebracht op de straf.",
    "Op basis van de ontvangen informatie uit het vergelijkbare geval wordt de verdachte schuldig bevonden aan poging tot zware mishandeling en bedreiging van ambtenaren met zware mishandeling. De verdachte wordt veroordeeld tot een gevangenisstraf van 24 maanden.",
    "Op basis van de ontvangen informatie uit het vergelijkbare geval wordt de verdachte schuldig bevonden aan mishandeling en bedreiging van ambtenaren in functie. De verdachte wordt veroordeeld tot een gevangenisstraf van 6 maanden, met een proeftijd van 2 jaar. Daarnaast dient de verdachte een geldboete van €500,- te betalen",
    "Op basis van de beschikbare context en de uitspraak in een vergelijkbare zaak, wordt de verdachte schuldig bevonden aan poging tot zware mishandeling door op 16 september 2020 te Lelystad het slachtoffer in het gezicht te snijden met een mes. De strafmaat wordt vastgesteld op een gevangenisstraf van 18 maanden, met de mogelijkheid dat deze straf niet ten uitvoer zal worden gelegd, tenzij de veroordeelde binnen een proeftijd van 3 jaren opnieuw een strafbaar feit pleegt.",
    "Op basis van de beschikbare context en de uitspraak in een vergelijkbare zaak, wordt de verdachte schuldig bevonden aan poging tot zware mishandeling door op 16 september 2020 te Lelystad het slachtoffer in het gezicht te snijden met een mes. De strafmaat wordt vastgesteld op een gevangenisstraf van 18 maanden, met de mogelijkheid dat deze straf niet ten uitvoer zal worden gelegd, tenzij de veroordeelde binnen een proeftijd van 3 jaren opnieuw een strafbaar feit pleegt.",
    "Op basis van mijn kennis van het Nederlandse strafrecht wordt de verdachte schuldig bevonden aan mishandeling van het slachtoffer door hem met een mes in de wang te snijden. De verdachte wordt veroordeeld tot een taakstraf van 120 uur. Daarnaast wordt de verdachte veroordeeld voor bedreiging van de ambtenaren met een gevangenisstraf van 6 maanden.",
    "Op basis van de verstrekte informatie en de tenlastelegging aan de verdachte, luidt de beslissing als volgt: verklaart het onder 1 ten laste gelegde niet bewezen en spreekt verdachte daarvan vrij. Verklaart bewezen dat verdachte het onder 2 ten laste gelegde heeft begaan zoals hiervoor in rubriek 5 is vermeld. Verklaart niet bewezen wat aan verdachte meer of anders is ten laste gelegd dan hiervoor is bewezen verklaard en spreekt verdachte daarvan vrij. Het bewezen verklaarde levert op: Ten aanzien van feit 2: bedreiging met enig misdrijf tegen het leven gericht en/of met zware mishandeling van ambtenaren in functie. Verklaart het bewezene strafbaar. Verklaart verdachte, [verdachte], daarvoor strafbaar. Veroordeelt verdachte tot een gevangenisstraf van 6 maanden. Beveelt dat de tijd die door veroordeelde voor de tenuitvoerlegging van deze uitspraak in voorlopige hechtenis is doorgebracht, bij de tenuitvoerlegging van die straf in mindering gebracht zal worden.",
    "Op basis van de beschikbare context en de uitspraak in een vergelijkbare zaak, wordt de verdachte schuldig bevonden aan poging tot zware mishandeling door op 16 september 2020 te Lelystad het slachtoffer in het gezicht te snijden met een mes. De strafmaat wordt vastgesteld op een gevangenisstraf van 18 maanden, met de mogelijkheid dat deze straf niet ten uitvoer zal worden gelegd, tenzij de veroordeelde binnen een proeftijd van 3 jaren opnieuw een strafbaar feit pleegt.",
    "Op basis van de beschikbare context en de uitspraak in een vergelijkbare zaak, wordt de verdachte schuldig bevonden aan poging tot zware mishandeling door op 16 september 2020 te Lelystad het slachtoffer in het gezicht te snijden met een mes. De strafmaat wordt vastgesteld op een gevangenisstraf van 18 maanden, met de mogelijkheid dat deze straf niet ten uitvoer zal worden gelegd, tenzij de veroordeelde binnen een proeftijd van 3 jaren opnieuw een strafbaar feit pleegt.",
    "Op basis van de gegeven informatie in de vergelijkbare zaak, wordt de verdachte schuldig bevonden aan mishandeling en bedreiging met een misdrijf tegen het leven gericht. De strafmaat wordt vastgesteld op een gevangenisstraf van 6 maanden, met de mogelijkheid dat deze straf niet ten uitvoer zal worden gelegd, tenzij de veroordeelde binnen een proeftijd van 2 jaar opnieuw een strafbaar feit pleegt.",
    "Op basis van de gegeven context en de aanklachten tegen de verdachte, zou de mogelijke uitspraak als volgt kunnen luiden: Verklaart dat verdachte de ten laste gelegde feiten heeft begaan zoals beschreven. Het bewezen verklaarde levert op: \nPrimair: poging tot zware mishandeling. \nBedreiging van ambtenaren tijdens de rechtmatige uitoefening van hun bediening met enig misdrijf tegen het leven gericht en/of met zware mishandeling. Verklaart het bewezene strafbaar. Verklaart verdachte, [naam van de verdachte], daarvoor strafbaar. Veroordeelt verdachte tot een gevangenisstraf van 24 maanden voor feit 1 en 12 maanden voor feit 2, deze straffen zullen niet gelijktijdig ten uitvoer worden gelegd. Beveelt dat de tijd door verdachte voor de tenuitvoerlegging van dit vonnis in voorlopige hechtenis is doorgebracht, bij de uitvoering van deze straf in mindering zal worden gebracht. Dit vonnis is gewezen door mr. [naam rechter], voorzitter, mrs. [naam rechter] en [naam rechter], rechters, in tegenwoordigheid van [naam griffier], griffier, en uitgesproken op de openbare terechtzitting van deze rechtbank.",
    "Op basis van de nieuwe tenlastelegging zou de mogelijke uitspraak als volgt kunnen luiden: Verklaart het onder 1 ten laste gelegde niet bewezen en spreekt verdachte daarvan vrij. Verklaart bewezen dat verdachte het onder 2 ten laste gelegde heeft begaan, namelijk medeplegen van opzettelijk gebruik maken van een vals geschrift. Het bewezen verklaarde levert op: medeplegen van opzettelijk gebruik maken van een vals geschrift, als bedoeld in artikel 225, eerste lid, van het Wetboek van Strafrecht, als ware het echt en onvervalst, meermalen gepleegd. Verklaart het bewezene strafbaar en veroordeelt verdachte tot een gevangenisstraf van 4 maanden, met een proeftijd van 1 jaar. De tenuitvoerlegging kan worden gelast indien de veroordeelde zich voor het einde van de proeftijd schuldig maakt aan een strafbaar feit.",
    "Op basis van de nieuwe tenlastelegging zou de mogelijke uitspraak als volgt kunnen luiden: Verklaart het onder 1 ten laste gelegde niet bewezen en spreekt verdachte daarvan vrij. Verklaart bewezen dat verdachte het onder 2 ten laste gelegde heeft begaan, namelijk medeplegen van opzettelijk gebruik maken van een vals geschrift. Het bewezen verklaarde levert op: medeplegen van opzettelijk gebruik maken van een vals geschrift, als bedoeld in artikel 225, eerste lid, van het Wetboek van Strafrecht, als ware het echt en onvervalst, meermalen gepleegd. Verklaart het bewezene strafbaar en veroordeelt verdachte tot een gevangenisstraf van 4 maanden, met een proeftijd van 1 jaar. De tenuitvoerlegging kan worden gelast indien de veroordeelde zich voor het einde van de proeftijd schuldig maakt aan een strafbaar feit.",
    "Op basis van de verstrekte informatie wordt de verdachte schuldig bevonden aan poging tot zware mishandeling. De straf die wordt opgelegd is een gevangenisstraf van 18 maanden, met een proeftijd van 3 jaar. De tenuitvoerlegging van de straf zal niet plaatsvinden, tenzij de veroordeelde binnen de proeftijd opnieuw een strafbaar feit pleegt.",
    "Op basis van de verstrekte informatie wordt de verdachte schuldig bevonden aan mishandeling en bedreiging met een misdrijf tegen het leven gericht en zware mishandeling. De opgelegde straf is een gevangenisstraf van 12 maanden, met een proeftijd van 3 jaar. De tenuitvoerlegging van de straf zal niet plaatsvinden, tenzij de veroordeelde binnen de proeftijd opnieuw een strafbaar feit pleegt.",
    "Op basis van het Nederlandse Wetboek van Strafrecht wordt de verdachte schuldig bevonden aan een poging tot zware mishandeling en bedreiging van ambtenaren in functie met een misdrijf tegen het leven gericht. Gelet op de ernst van de feiten en het gevaar dat de verdachte vormt, wordt een gevangenisstraf van 24 maanden opgelegd, met een proeftijd van 3 jaar. Daarnaast kunnen bijzondere voorwaarden worden opgelegd, zoals een meldplicht bij de reclassering, het volgen van een behandeling en het verbod op het bezit van wapens. Dit vonnis zou worden uitgesproken op een openbare terechtzitting van de rechtbank, conform de gangbare procedure in Nederland.",
    "Op basis van het Nederlandse Wetboek van Strafrecht wordt de verdachte schuldig bevonden aan een poging tot zware mishandeling en bedreiging van ambtenaren in functie met een misdrijf tegen het leven gericht. Gelet op de ernst van de feiten en het gevaar dat de verdachte vormt, wordt een gevangenisstraf van 24 maanden opgelegd, met een proeftijd van 3 jaar. Daarnaast kunnen bijzondere voorwaarden worden opgelegd, zoals een meldplicht bij de reclassering, het volgen van een behandeling en het verbod op het bezit van wapens.",
    "Op basis van het Nederlandse Wetboek van Strafrecht wordt de verdachte schuldig bevonden aan een poging tot zware mishandeling en bedreiging van ambtenaren in functie met een misdrijf tegen het leven gericht. Gelet op de ernst van de feiten en het gevaar dat de verdachte vormt, wordt een gevangenisstraf van 24 maanden opgelegd, met een proeftijd van 3 jaar. Daarnaast kunnen bijzondere voorwaarden worden opgelegd, zoals een meldplicht bij de reclassering, het volgen van een behandeling en het verbod op het bezit van wapens.",
    "Op basis van de verstrekte informatie en mijn kennis van het Nederlandse rechtssysteem, wordt de verdachte vrijgesproken van het onder 1 ten laste gelegde. Het onder 2 ten laste gelegde wordt bewezen geacht en de verdachte wordt veroordeeld voor bedreiging van ambtenaren in functie. De straf die wordt opgelegd is een gevangenisstraf van 9 maanden.",
    "Op basis van de verstrekte informatie en mijn kennis van het Nederlandse rechtssysteem, wordt de verdachte vrijgesproken van het onder 1 ten laste gelegde. Het onder 2 ten laste gelegde wordt bewezen geacht en de verdachte wordt veroordeeld voor bedreiging van ambtenaren in functie. De straf die wordt opgelegd is een gevangenisstraf van 6 maanden.",
    "Op basis van mijn kennis van het Nederlandse rechtssysteem, zou de beslissing van de rechtbank kunnen zijn als volgt: De rechtbank verklaart bewezen dat verdachte het onder 1 primair en subsidiair ten laste gelegde heeft begaan, namelijk het opzettelijk snijden in het gezicht van het slachtoffer met een mes of scherp voorwerp met het oogmerk zwaar lichamelijk letsel toe te brengen, waarbij de uitvoering van het voorgenomen misdrijf niet is voltooid. Ten aanzien van feit 2 wordt bewezen verklaard dat verdachte ambtenaren heeft bedreigd met enig misdrijf tegen het leven gericht of zware mishandeling door met een mes vanaf een korte afstand op hen af te rennen en dreigende bewegingen te maken. \nHet bewezen verklaarde levert op: \nTen aanzien van feit 1 primair en subsidiair: poging tot zware mishandeling. \nTen aanzien van feit 2: bedreiging met geweld tegen ambtenaren in functie. \nVerklaart het bewezene strafbaar en veroordeelt verdachte, [naam van de verdachte], daarvoor strafbaar. Veroordeelt verdachte tot een gevangenisstraf van 24 maanden. Daarnaast legt de rechtbank een contact- en locatieverbod op ten aanzien van de slachtoffers en de ambtenaren. Dit vonnis is gewezen door mr. [naam rechter], voorzitter, mrs. [naam rechter] en [naam rechter], rechters, in tegenwoordigheid van [naam griffier], griffier, en uitgesproken op de openbare terechtzitting van deze rechtbank.",
    "Op basis van de informatie uit de eerder vergelijkbare zaak wordt de verdachte schuldig bevonden aan de ten laste gelegde feiten. De straf die wordt opgelegd is een gevangenisstraf van vier jaar. Daarnaast zal de tijd die de verdachte al in voorarrest heeft gezeten worden afgetrokken van de totale straf. Bepaalde items worden verbeurd verklaard als aanvullende straf voor de gepleegde misdrijven. Het is belangrijk op te merken dat deze informatie gebaseerd is op de context van de eerder vergelijkbare zaak en geen specifiek juridisch advies of interpretatie bevat.",
    "Op basis van de verstrekte informatie zou de beslissing zijn: Verklaart bewezen dat verdachte het onder 1 subsidiair en 2 ten laste gelegde heeft begaan zoals hiervoor is vermeld. Verklaart niet bewezen wat aan verdachte meer of anders is ten laste gelegd dan hiervoor bewezen is verklaard en spreekt verdachte daarvan vrij. Het bewezen verklaarde levert op: \nTen aanzien van feit 1 subsidiair: mishandeling. \nTen aanzien van feit 2: bedreiging van een ambtenaar tijdens de uitoefening van zijn functie. Verklaart het bewezene strafbaar. Verklaart verdachte, [naam van de verdachte], daarvoor strafbaar. Veroordeelt verdachte tot een gevangenisstraf van 6 maanden. Beveelt dat de tijd die door veroordeelde voor de tenuitvoerlegging van deze uitspraak in voorlopige hechtenis is doorgebracht, bij de tenuitvoerlegging van die straf in mindering zal worden gebracht.",
    "Op basis van de verstrekte informatie wordt de verdachte schuldig bevonden aan een poging tot zware mishandeling en bedreiging van ambtenaren met een mes. De rechtbank veroordeelt de verdachte tot een gevangenisstraf van 24 maanden voor de poging tot zware mishandeling en een gevangenisstraf van 6 maanden voor de bedreiging van ambtenaren. Deze straffen zullen gelijktijdig ten uitvoer worden gelegd.",
    "Op basis van de beschikbare informatie wordt de verdachte vrijgesproken van het onder 1 ten laste gelegde feit. Voor het onder 2 ten laste gelegde feit, namelijk bedreiging van een ambtenaar in functie met enig misdrijf tegen het leven gericht en/of met zware mishandeling, wordt de verdachte schuldig bevonden. De rechtbank veroordeelt de verdachte tot een gevangenisstraf van 18 maanden.",
    "Op basis van de verstrekte informatie uit de eerdere, vergelijkbare zaak wordt de verdachte vrijgesproken van het eerste ten laste gelegde feit en schuldig bevonden aan het tweede ten laste gelegde feit, namelijk bedreiging van ambtenaren tijdens de rechtmatige uitoefening van hun bediening. De verdachte wordt veroordeeld tot een gevangenisstraf van 24 maanden. Deze straf wordt bepaald op basis van de ernst van de feiten en de persoonlijke omstandigheden van de verdachte. Daarnaast kan er een contactverbod met de slachtoffers en eventueel een schadevergoeding worden opgelegd.",
    "Op basis van de informatie uit de vergelijkbare zaak wordt de verdachte vrijgesproken van het eerste ten laste gelegde feit en schuldig bevonden aan het tweede ten laste gelegde feit, namelijk bedreiging van ambtenaren tijdens de rechtmatige uitoefening van hun bediening. De verdachte wordt veroordeeld tot een gevangenisstraf van 6 maanden. Deze straf wordt bepaald op basis van de ernst van de feiten en de persoonlijke omstandigheden van de verdachte. Daarnaast kan er een contactverbod met de slachtoffers en eventueel een schadevergoeding worden opgelegd.",
    "Op basis van de vergelijkbare zaken wordt de verdachte schuldig bevonden aan het tweede ten laste gelegde feit, namelijk bedreiging van ambtenaren tijdens de rechtmatige uitoefening van hun bediening. De verdachte wordt veroordeeld tot een gevangenisstraf van 6 maanden. Daarnaast kan er een contactverbod met de slachtoffers en eventueel een schadevergoeding worden opgelegd.",
    "Op basis van de gegeven context wordt de verdachte schuldig bevonden aan poging tot zware mishandeling en bedreiging van ambtenaren met enig misdrijf tegen het leven gericht en zware mishandeling. De verdachte wordt veroordeeld tot een gevangenisstraf van 24 maanden. Daarnaast kan er een contactverbod met de slachtoffers en eventueel een schadevergoeding worden opgelegd.",
    "De verdachte wordt schuldig bevonden aan poging tot zware mishandeling en bedreiging van ambtenaren. Op basis van de ernst van de aanklachten en de bewezen verklaringen, kan de verdachte veroordeeld worden tot een gevangenisstraf van 24 maanden.",
    "Op basis van de Nederlandse wetgeving en de tenlastelegging die aan de verdachte is voorgelegd, zou het vonnis als volgt kunnen luiden: \nDe rechtbank verklaart bewezen dat de verdachte: \nOp 16 september 2020 te Lelystad, ter uitvoering van het door de verdachte voorgenomen misdrijf om aan [slachtoffer] opzettelijk zwaar lichamelijk letsel toe te brengen, opzettelijk naar voornoemde [slachtoffer] is gegaan, waarna hij, de verdachte, voornoemde [slachtoffer] met een mes en/of een stuk glas heeft gesneden in de wang, terwijl de uitvoering van dat voorgenomen misdrijf niet is voltooid. \nOp 16 september 2020 te Lelystad, één of meer ambtenaren, te weten [verbalisant 1], [verbalisant 3] en [verbalisant 4], gedurende en ter zake van de rechtmatige uitoefening van hun bediening heeft bedreigd met enig misdrijf tegen het leven gericht en/of met zware mishandeling, door met een mes vanaf een korte afstand op voornoemde ambtenaren af te rennen en maaiende bewegingen te maken met zijn handen. \nDe rechtbank verklaart de verdachte schuldig aan de ten laste gelegde feiten en acht deze strafbaar. Veroordeelt de verdachte tot een gevangenisstraf van 24 maanden.",
    "Op basis van de ontvangen context en de eerdere uitspraak in een soortgelijke zaak, luidt het vonnis als volgt: \nDe rechtbank verklaart bewezen dat de verdachte: \nOp 16 september 2020 te Lelystad, [slachtoffer] heeft mishandeld door voornoemde [slachtoffer] met een mes en/of een stuk glas te snijden in de wang, in het gezicht. \nOp 16 september 2020 te Lelystad, één ambtenaar, te weten [verbalisant 1], gedurende en ter zake van de rechtmatige uitoefening van zijn bediening heeft bedreigd met enig misdrijf tegen het leven gericht en/of met zware mishandeling. \nDe rechtbank verklaart de verdachte schuldig aan de ten laste gelegde feiten en acht deze strafbaar. Veroordeelt de verdachte tot een gevangenisstraf van 10 maanden.",
    "Op basis van mijn kennis van het Nederlandse rechtssysteem, zou ik de volgende beslissing nemen in deze zaak: \nVerklaart ten aanzien van feit 1 dat verdachte schuldig is aan poging tot zware mishandeling van het slachtoffer op 16 september 2020 te Lelystad. Verklaart ten aanzien van feit 2 dat verdachte schuldig is aan het bedreigen van ambtenaren gedurende de rechtmatige uitoefening van hun bediening met enig misdrijf tegen het leven gericht en/of met zware mishandeling op dezelfde datum en locatie. Het bewezen verklaarde levert op: poging tot zware mishandeling en bedreiging van ambtenaren. \nVerklaart het bewezene strafbaar. Verklaart verdachte, [naam van de verdachte], daarvoor strafbaar. Veroordeelt verdachte tot een gevangenisstraf van 24 maanden. Beveelt dat de tijd die door veroordeelde voor de tenuitvoerlegging van deze uitspraak in voorlopige hechtenis is doorgebracht, bij de tenuitvoerlegging van die straf in mindering gebracht zal worden. \nDit is een hypothetische beslissing op basis van de gegeven tenlastelegging. Voor een nauwkeurigere beslissing, raad ik aan om een advocaat te raadplegen of het vonnis van de rechter af te wachten.",
    "Op basis van de verstrekte informatie wordt de verdachte schuldig bevonden aan poging tot zware mishandeling en bedreiging van ambtenaren. De verdachte wordt veroordeeld tot een gevangenisstraf van 24 maanden. De tijd die de verdachte in voorarrest heeft doorgebracht, zal in mindering worden gebracht op de opgelegde straf.",
    "Op basis van het Nederlandse rechtssysteem en de gegeven context, wordt de verdachte vrijgesproken van het onder 1 ten laste gelegde. De verdachte wordt echter schuldig bevonden aan het onder 2 ten laste gelegde, namelijk bedreiging met enig misdrijf tegen het leven gericht en met zware mishandeling van ambtenaren gedurende en ter zake van de rechtmatige uitoefening van hun bediening. De verdachte wordt veroordeeld tot een gevangenisstraf van 24 maanden.",
    "Op basis van mijn kennis over het Nederlandse rechtssysteem, zou de beslissing van de rechtbank voor de tenlastelegging aan de verdachte als volgt kunnen luiden: \nDe rechtbank verklaart bewezen dat verdachte: \nOp 16 september 2020 te Lelystad, ter uitvoering van het door verdachte voorgenomen misdrijf om aan [slachtoffer] opzettelijk zwaar lichamelijk letsel toe te brengen, opzettelijk naar voornoemde [slachtoffer] is gegaan, waarna hij voornoemde [slachtoffer] met een mes en/of een stuk glas heeft gesneden in de wang, in het gezicht, terwijl de uitvoering van dat voorgenomen misdrijf niet is voltooid. \nOp 16 september 2020 te Lelystad, één of meer ambtenaren, te weten [verbalisant 1], [verbalisant 3], en [verbalisant 4], gedurende en ter zake van de rechtmatige uitoefening van hun bediening heeft bedreigd met enig misdrijf tegen het leven gericht en/of met zware mishandeling, door met een mes vanaf een korte afstand op voornoemde ambtenaren af te rennen en (daarbij) maaiende en korte bewegingen te maken met zijn handen. \nVerklaart het bewezene strafbaar en veroordeelt verdachte, [verdachte], daarvoor strafbaar. Veroordeelt verdachte tot een gevangenisstraf van 24 maanden. Dit vonnis is gewezen door [naam rechters], in aanwezigheid van [naam griffier], en uitgesproken op de openbare terechtzitting van deze rechtbank.",
    "Op basis van de informatie die ik heb ontvangen en mijn kennis over het Nederlandse rechtssysteem, zou de verdachte in dit geval veroordeeld kunnen worden voor het opzettelijk toebrengen van zwaar lichamelijk letsel en het bedreigen van ambtenaren. \nVerdict: \nVoor het opzettelijk toebrengen van zwaar lichamelijk letsel en bedreigen van ambtenaren zou de verdachte veroordeeld kunnen worden tot een gevangenisstraf van 24 maanden.",
    "Op basis van de tenlastelegging die je hebt gegeven, zou de verdachte veroordeeld kunnen worden voor poging tot zware mishandeling en bedreiging van ambtenaren. \nVerdict: \nVoor de poging tot zware mishandeling zou de verdachte veroordeeld kunnen worden tot een gevangenisstraf van 12 maanden, waarvan 6 maanden voorwaardelijk, met een proeftijd van 2 jaar. \nVoor de bedreiging van ambtenaren zou de verdachte veroordeeld kunnen worden tot een gevangenisstraf van 6 maanden, met een proeftijd van 2 jaar.",
    "Op basis van de context die is gegeven, zou de verdachte veroordeeld kunnen worden voor mishandeling en bedreiging van ambtenaren. \nVerdict op basis van de context: \nVoor de mishandeling zou de verdachte veroordeeld kunnen worden tot een gevangenisstraf van 24 maanden.",
    "Op basis van de eerder gegeven context en de uitspraak van de rechter in een vergelijkbare zaak, zou de verdachte schuldig worden bevonden aan een poging tot zware mishandeling en bedreiging met enig misdrijf tegen het leven gericht. De straf die daarbij hoort is een gevangenisstraf van 24 maanden.",
    "Op basis van de eerder gegeven context en de uitspraak van de rechter in een vergelijkbare zaak, zou de verdachte schuldig worden bevonden aan mishandeling en bedreiging met een misdrijf tegen het leven gericht. De straf die daarbij hoort is een gevangenisstraf van 6 maanden.",
    "Op basis van de context en de beschuldiging aan de verdachte zou de rechtbank kunnen beslissen dat de verdachte schuldig is aan poging tot zware mishandeling en bedreiging van een ambtenaar in functie. Daarom wordt de verdachte veroordeeld tot een gevangenisstraf van 24 maanden voor het eerste feit en 6 maanden voor het tweede feit, in totaal 30 maanden gevangenisstraf.",
    "Op basis van de gegeven context wordt de verdachte vrijgesproken van het eerste ten laste gelegde feit. Voor het tweede ten laste gelegde feit, namelijk bedreiging van een ambtenaar tijdens de uitoefening van zijn functie, wordt de verdachte schuldig bevonden en veroordeeld tot een gevangenisstraf van 6 maanden. Deze veroordeling is gebaseerd op de eerder behandelde zaak en de strafmaat die daarbij is opgelegd.",
    "Op basis van de gegeven context wordt de verdachte vrijgesproken van het voornemen om zwaar lichamelijk letsel toe te brengen aan het slachtoffer, aangezien de uitvoering van dat voorgenomen misdrijf niet is voltooid. Echter, de verdachte wordt schuldig bevonden aan het snijden van het slachtoffer met een mes in de wang en het bedreigen van ambtenaren met een misdrijf tegen het leven gericht en/of met zware mishandeling. Daarom wordt de verdachte veroordeeld tot een gevangenisstraf van 6 maanden.",
    "Op basis van de informatie uit de vergelijkbare zaak wordt de verdachte vrijgesproken van het eerste ten laste gelegde feit en schuldig bevonden aan het tweede ten laste gelegde feit. De verdachte wordt veroordeeld tot een gevangenisstraf van 6 maanden. Daarnaast kan er een contactverbod met de slachtoffers en eventueel een schadevergoeding worden opgelegd."   
]

P, R, F1 = bert_score(hypotheses, references, lang="en", verbose=True)
mean_bert_f1 = F1.mean().item()
variance_bert_f1 = np.var(F1.tolist())
std_dev_bert_f1 = np.std(F1.tolist())

print(f"Mean BERTScore F1: {mean_bert_f1}")
print(f"Variance of BERTScore F1: {variance_bert_f1}")
print(f"Standard Deviation of BERTScore F1: {std_dev_bert_f1}")

rouge = Rouge()
rouge_scores = rouge.get_scores(hypotheses, references, avg=True)

mean_rouge_1 = rouge_scores['rouge-1']['f']
mean_rouge_2 = rouge_scores['rouge-2']['f']
mean_rouge_l = rouge_scores['rouge-l']['f']

print(f"Mean ROUGE-1 F1: {mean_rouge_1}")
print(f"Mean ROUGE-2 F1: {mean_rouge_2}")
print(f"Mean ROUGE-L F1: {mean_rouge_l}")





tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 147.65 seconds, 0.33 sentences/sec
Mean BERTScore F1: 0.8756052255630493
Variance of BERTScore F1: 0.0013176033055742717
Standard Deviation of BERTScore F1: 0.03629880584226252
Mean ROUGE-1 F1: 0.4502739618943695
Mean ROUGE-2 F1: 0.2622152932737372
Mean ROUGE-L F1: 0.4216564530865912
